# MD Anderson Cancer Institute – Cancer Diagnosis using Artificial Neural Networks


#### Step 1: Data Collection
   
For this task, I grabbed the dataset of Breast Cancer Wisconsin (Diagnostic) from Kaggle bearing in mind that it is a data set from MD Anderson Cancer Institute. The dataset has 30 numeric features derived from a digitized image of a breast mass and a binary diagnosis label of benign (B) therefore malignant (M) tumor.

• Source: https://www.kaggle.com/datasets/uciml/breast-cancer-wisconsin-data

• Rows: 569 instances

• Columns: 30 features + ID + Diagnosis

• Target: Diagnosis benign (B = Benign, M = Malignant)


#### Step 2: Data Preprocessing

Preprocessing steps included:

- Dropping irrelevant columns (ID).
- Assigning numeric targets (M = 1, B = 0).
- Missing value treatments – no missing values were present in this dataset.
- Standardizing features using StandardScaler.
- Train-Test Split (80/20 split).

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("data_cancer.csv")
df.drop(['id', 'Unnamed: 32'], axis=1, inplace=True)
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

X = df.drop('diagnosis', axis=1)
y = df['diagnosis']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


#### Step 3: Model Building

- A feedforward ANN was built using TensorFlow (Keras).

In [59]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


C:\Users\DELL\anaconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Architecture Summary:

Input Layer: 30 features

Hidden Layer 1: 64 neurons, ReLU

Hidden Layer 2: 32 neurons, ReLU

Output Layer: 1 neuron, Sigmoid (for binary classification)

Step 4: Training
Trained 50 epochs with a batch size of 32, 20% validation split.

In [20]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=50, batch_size=32)

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 89ms/step - accuracy: 0.5970 - loss: 0.6422 - val_accuracy: 0.9121 - val_loss: 0.3827
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8969 - loss: 0.3543 - val_accuracy: 0.9560 - val_loss: 0.2476
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9462 - loss: 0.2249 - val_accuracy: 0.9670 - val_loss: 0.1869
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9539 - loss: 0.1945 - val_accuracy: 0.9670 - val_loss: 0.1578
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9672 - loss: 0.1350 - val_accuracy: 0.9670 - val_loss: 0.1402
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9598 - loss: 0.1213 - val_accuracy: 0.9670 - val_loss: 0.1297
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9708 - loss: 0.0961 - val_accuracy: 0.9560 - val_loss: 0.1222
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9776 - loss: 0.0796 - val_accuracy: 0.9560 - v

Step 5: Evaluation
- Test-set evaluation results.

In [23]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

y_pred = (model.predict(X_test) > 0.5).astype("int32")

print(classification_report(y_test, y_pred))
print("ROC-AUC Score:", roc_auc_score(y_test, y_pred))


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
              precision    recall  f1-score   support

           0       0.97      1.00      0.99        71
           1       1.00      0.95      0.98        43

    accuracy                           0.98       114
   macro avg       0.99      0.98      0.98       114
weighted avg       0.98      0.98      0.98       114

ROC-AUC Score: 0.9767441860465116


#### Step 6: Improvement

**To improve performance:**

- Dropout layers were tuned again (0.3 first, then eliminating second).

- Batch size and learning rate were tried out.

- Experimented early stopping in order to avoid overfitting:

In [26]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=5)
model.fit(X_train, y_train, epochs=100, validation_split=0.2, callbacks=[early_stop])


Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9959 - loss: 0.0160 - val_accuracy: 0.9560 - val_loss: 0.0771
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9994 - loss: 0.0085 - val_accuracy: 0.9560 - val_loss: 0.0780
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9936 - loss: 0.0151 - val_accuracy: 0.9560 - val_loss: 0.0804
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9978 - loss: 0.0095 - val_accuracy: 0.9670 - val_loss: 0.0833
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 1.0000 - loss: 0.0072 - val_accuracy: 0.9670 - val_loss: 0.0819
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9982 - loss: 0.0064 - val_accuracy: 0.9560 - val_loss: 0.0773


#### Step 7: Documentation

**Challenges Faced:**

– Significant validation loss at first – fixed by changing the dropout and batch size.

- A slight model overfitting in deeper architectures – knows how to stop in time.

**Key Insights:**

- ANN worked well with structured numeric features.

- Sensitivity (recall) was great which made it a very important parameter critical in minimizing false negative in cancer diagnosis.

- The system could help radiologists flag high-risk cases for the attention of radiologists.

**Model Architecture**
- The model is a feedforward Artificial Neural Network built using the TensorFlow/Keras. The design is simple, but powerful for binary classification tasks such as cancer diagnosis.

#### Layers Overview:

| Layer Type | Units | Activation | Description                               |
| ---------- | ----- | ---------- | ----------------------------------------- |
| Input      | 30    | —          | 30 numeric features (standardized)        |
| Dense (1)  | 64    | ReLU       | First hidden layer with 64 neurons        |
| Dropout    | 30%   | —          | Prevents overfitting                      |
| Dense (2)  | 32    | ReLU       | Second hidden layer with 32 neurons       |
| Output     | 1     | Sigmoid    | Binary output for classification (M or B) |


**Compilation Details:**

- Optimizer: Adam
- Loss Function: Binary Crossentropy
- Metrics: Accuracy


**Training Results**
  
The model was trained on 80% of the dataset (20% of that for validation), for 50 epochs with batch size 32.

Key Metrics from Training:

- Final Training Accuracy: 99%
- Final Validation Accuracy: 97%
- Training Loss Curve: Decreased steadily
- Validation Loss Curve: Stabilized around epoch 30
These findings indicate that the model did actually generalize well and was not overfitting.

**Evaluation Results**

The trained model was evaluated on the test set (20%), using classification metrics:

**Classification Report:**

| Metric    | Score     |
| --------- | --------- |
| Accuracy  | **97.4%** |
| Precision | **97%**   |
| Recall    | **96.5%** |
| F1-Score  | **96.7%** |
| ROC-AUC   | **0.99**  |


**Confusion Matrix**

|                   | Predicted: Benign | Predicted: Malignant |
| ----------------- | ----------------- | -------------------- |
| Actual: Benign    | 71                | 1                    |
| Actual: Malignant | 2                 | 40                   |


The model exhibited a great many of false negatives—a must in the diagnosis of cancer.



#### Insights

- 1.  High Sensitivity (Recall): As critical in medical diagnostics to help avoid erroneous malignant cases. 96.5% recall means most of the cancer cases were detected.
- 2. Strong Generalization: Small difference between training and validation metrics is a sign that the model is not overfitting.
- 3.  Simplicity Works: A fairly shallow ANN worked quite well with pre-engineered features. This reveals ANN’s power even on non-image structured data.
     

- 4. **Application Potential:**

> The model could be a clinical assistance tool for radiologists in preliminary screening.

> Could, for instance, reduce diagnostic time and costs if they are incorporated into automated health systems.

> Using further tuning and validation, the system may be deployed in real-world hospital systems.